# SD212: Graph mining

## Lab 7: Graph neural networks


In this lab, you will learn to train and use graph neural networks (GNN).

## Import

In [188]:
import numpy as np
from scipy import sparse

In [189]:
import dgl
from dgl.nn import SAGEConv
from dgl import function as fn

In [190]:
import torch
from torch import nn
import torch.nn.functional as F

In [191]:
from sknetwork.data import load_netset
from sknetwork.classification import DirichletClassifier
from sknetwork.embedding import Spectral

In [192]:
from sklearn.metrics import f1_score

## Load data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Cora (directed graph + bipartite graph)
* WikiVitals (directed graph + bipartite graph)

Both graphs have ground-truth labels.

In [193]:
cora = load_netset('cora')

Parsing files...
Done.


In [194]:
wikivitals = load_netset('wikivitals')

Parsing files...
Done.


## Split data

As GNN must be trained on a specific task (e.g., node classification), we need to split the set of nodes.

In [195]:
def split_train_test_val(n_nodes, test_ratio=0.1, val_ratio=0.1, seed=None):
    """Split the nodes into train / test / validation sets.
    
    Parameters
    ----------
    n_nodes: int
        Number of nodes.
    test_ratio: float
        Test ratio.
    val_ratio: float
        Validation ratio.
        The sum of test_ratio and validation_ratio cannot exceed 1.
    seed: int
        Random seed.
        
    Returns
    -------
    train: numpy array
        Boolean mask (train set)
    test: numpy array
        Boolean mask (test set)
    val: numpy array
        Boolean mask (validation set)
    """
    if seed:
        np.random.seed(seed)

    # test
    index = np.random.choice(n_nodes, int(np.ceil(n_nodes * test_ratio)), replace=False)
    test = np.zeros(n_nodes, dtype=bool)
    test[index] = 1
    
    # validation
    index = np.random.choice(np.argwhere(~test).ravel(), int(np.ceil(n_nodes * val_ratio)), replace=False)
    val = np.zeros(n_nodes, dtype=bool)
    val[index] = 1
    
    # train
    train = np.ones(n_nodes, dtype=bool)
    train[test] = 0
    train[val] = 0
    return train, test, val

## Tensors

The Deep Graph Library is based on PyTorch and uses tensors.

We here transform the Cora dataset into the appropriate format.

In [196]:
# citations between scientific papers
adjacency = cora.adjacency
# category of papers
labels = cora.labels
# using keywords of scientific papers as features
features = cora.biadjacency

In [197]:
np.unique(labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]), array([351, 217, 418, 818, 426, 298, 180]))

In [198]:
graph = dgl.from_scipy(adjacency)

In [199]:
type(graph)

dgl.heterograph.DGLHeteroGraph

In [200]:
labels = torch.Tensor(labels).long()
features = torch.Tensor(features.toarray())

In [201]:
labels.shape

torch.Size([2708])

In [202]:
features.shape

torch.Size([2708, 1433])

In [203]:
n_nodes = len(labels)
train, test, val = split_train_test_val(n_nodes)

In [204]:
train = torch.Tensor(train).bool()
test = torch.Tensor(test).bool()
val = torch.Tensor(val).bool()

## Graph neural networks

Here is a simple graph neural network with 2 layers and a ReLU activation function inbetween.

In [205]:
class GNN(nn.Module):
    def __init__(self, dim_input, dim_output, dim_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv(dim_input, dim_hidden, 'mean')
        self.conv2 = SAGEConv(dim_hidden, dim_output, 'mean')
        
    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

You can check the layer SAGEConv on the documentation

In [206]:
def init_model(model, features, labels):
    dim_input = features.shape[1]
    dim_output = len(labels.unique())
    return model(dim_input, dim_output)   

In [207]:
gnn = init_model(GNN, features, labels)

In [208]:
# prediction before training
output = gnn(graph, features)

In [209]:
output.shape

torch.Size([2708, 7])

In [210]:
def eval_model(gnn, graph, features, labels, test):
    gnn.eval()
    with torch.no_grad():
        output = gnn(graph, features)
        labels_pred = torch.max(output, dim=1)[1]
        score = f1_score(np.array(labels[test]), np.array(labels_pred[test]), average='micro')
    return score

In [211]:
def train_model(gnn, graph, features, labels, train, val, n_epochs=100, verbose=False):
    
    optimizer = torch.optim.Adam(gnn.parameters(), lr=0.01)
    
    gnn.train()
    scores = []
    
    for t in range(n_epochs):   
        
        # forward
        output = gnn(graph, features)
        logp = nn.functional.log_softmax(output, 1)
        loss = nn.functional.nll_loss(logp[train], labels[train])

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # evaluation
        score = eval_model(gnn, graph, features, labels, val)
        scores.append(score)
        
        if verbose and t % 5 == 0:
            print("Epoch {:02d} | Loss {:.3f} | Score {:.3f}".format(t, loss.item(), score))

## To do

* Train the GNN and give the F1 score of the classification on the test set.
* Compare with the score of a classification based on heat diffusion, using the graph only.

In [212]:
train_model(gnn, graph, features, labels, train, val, verbose=True)

Epoch 00 | Loss 2.074 | Score 0.399
Epoch 05 | Loss 0.802 | Score 0.768
Epoch 10 | Loss 0.376 | Score 0.852
Epoch 15 | Loss 0.201 | Score 0.875
Epoch 20 | Loss 0.103 | Score 0.863
Epoch 25 | Loss 0.058 | Score 0.871
Epoch 30 | Loss 0.038 | Score 0.882
Epoch 35 | Loss 0.025 | Score 0.878
Epoch 40 | Loss 0.018 | Score 0.882
Epoch 45 | Loss 0.013 | Score 0.882
Epoch 50 | Loss 0.010 | Score 0.878
Epoch 55 | Loss 0.008 | Score 0.882
Epoch 60 | Loss 0.007 | Score 0.882
Epoch 65 | Loss 0.006 | Score 0.886
Epoch 70 | Loss 0.005 | Score 0.882
Epoch 75 | Loss 0.005 | Score 0.875
Epoch 80 | Loss 0.004 | Score 0.875
Epoch 85 | Loss 0.004 | Score 0.875
Epoch 90 | Loss 0.003 | Score 0.875
Epoch 95 | Loss 0.003 | Score 0.875


In [213]:
eval_model(gnn, graph, features, labels, test)

0.8228782287822878

In [214]:
labels.unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5, 6]), tensor([351, 217, 418, 818, 426, 298, 180]))

In [215]:
# Get x seed per cluster
clusters = [np.argwhere(labels==label).ravel() for label in set(labels.numpy())]
n_seeds = 150
seeds = {i: labels.numpy()[i] for cluster in clusters for i in np.random.choice(cluster, n_seeds, replace=False)}

In [216]:
dirichlet = DirichletClassifier()
labels_pred_diffusion = dirichlet.fit_transform(adjacency, seeds)
f1_score(np.array(labels[test]), np.array(labels_pred_diffusion[test]), average='micro')

0.6383763837638377

In [217]:
# Get random seeds
n_seeds = 1000
seeds = {i: labels.numpy()[i] for i in np.random.choice(range(len(labels)), n_seeds,  replace=False)}

In [218]:
dirichlet = DirichletClassifier()
labels_pred_diffusion = dirichlet.fit_transform(adjacency, seeds)
f1_score(np.array(labels[test]), np.array(labels_pred_diffusion[test]), average='micro')

0.7343173431734318

## To do

* Apply the previous GNN to an empty graph. What is your conclusion?
* Compare with the score of a classification based on heat diffusion, using the features only.

In [219]:
graph = dgl.graph(([], []), num_nodes=len(graph.nodes()))

In [220]:
train_model(gnn, graph, features, labels, train, val, verbose=False)

In [221]:
eval_model(gnn, graph, features, labels, test)

0.7785977859778597

In [222]:
dirichlet = DirichletClassifier()
labels_pred_diffusion = dirichlet.fit_transform(adjacency, seeds)
f1_score(np.array(labels[test]), np.array(labels_pred_diffusion[test]), average='micro')

0.7343173431734318

## Build your own GNN

You will know build your own GNN using `dgl` pre-built functions for message passing between nodes.

## To do

* Implement message passing between nodes in a convolutional layer (hint: use `update_all()` function).
* Stack your custom layer in a GNN.
* Train your GNN on graph and features and give the F1 score of the classification on the test set.

In [225]:
class ConvLayer(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(ConvLayer, self).__init__()
        self.linear = nn.Linear(dim_input, dim_output)
        
    def forward(self, graph, features):
        with graph.local_scope():
            h_node = features
            # aggregation of neighbors
            graph.ndata['node'] = features
            graph.update_all(fn.copy_src('node', 'message'), fn.mean('message', 'neighbor'))
            h_neighbor = graph.ndata['neighbor']
            # sum of embeddings (node + neighbors)
            return self.linear(h_node + h_neighbor)

In [226]:
class GNN(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = ConvLayer(n_input, n_hidden)
        self.conv2 = ConvLayer(n_hidden, n_output)
        
    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [ ]:
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=True)

In [228]:
eval_model(gnn, graph, features, labels, test)

0.7453874538745388

## From graph only

In some cases, there are no features available. An option is to consider one-hot encoding of the nodes.

## To do

* Train a GNN considering one-hot encoding of the nodes as features.
* Compare with the score of a classification using more complex features.

In [231]:
features

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [229]:
features = torch.eye(cora.adjacency.shape[0])
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=True)

Epoch 00 | Loss 1.931 | Score 0.148
Epoch 05 | Loss 1.829 | Score 0.365
Epoch 10 | Loss 1.636 | Score 0.365
Epoch 15 | Loss 1.369 | Score 0.365
Epoch 20 | Loss 1.054 | Score 0.365
Epoch 25 | Loss 0.738 | Score 0.365
Epoch 30 | Loss 0.468 | Score 0.365
Epoch 35 | Loss 0.272 | Score 0.365
Epoch 40 | Loss 0.151 | Score 0.365
Epoch 45 | Loss 0.084 | Score 0.365
Epoch 50 | Loss 0.049 | Score 0.365
Epoch 55 | Loss 0.031 | Score 0.365
Epoch 60 | Loss 0.022 | Score 0.365
Epoch 65 | Loss 0.016 | Score 0.365
Epoch 70 | Loss 0.013 | Score 0.362
Epoch 75 | Loss 0.011 | Score 0.351
Epoch 80 | Loss 0.009 | Score 0.343
Epoch 85 | Loss 0.008 | Score 0.343
Epoch 90 | Loss 0.008 | Score 0.354
Epoch 95 | Loss 0.007 | Score 0.354


In [230]:
eval_model(gnn, graph, features, labels, test)

0.2693726937269373

## Wikivitals

## To do

* Train a GNN on Wikivitals using spectral embedding of the bipartite graph as features.
* Compare with the classification based on heat diffusion (either on the directed graph or the bipartite graph).

In [232]:
adjacency = wikivitals.adjacency
biadjacency = wikivitals.biadjacency
labels = wikivitals.labels

In [233]:
adjacency

<10011x10011 sparse matrix of type '<class 'numpy.bool_'>'
	with 824999 stored elements in Compressed Sparse Row format>

In [234]:
graph = dgl.from_scipy(adjacency)

In [235]:
n_nodes = len(labels)
train, test, val = split_train_test_val(n_nodes)

train = torch.Tensor(train).bool()
test = torch.Tensor(test).bool()
val = torch.Tensor(val).bool()

In [236]:
# With all features
labels = torch.Tensor(labels).long()
features = torch.Tensor(biadjacency.todense())

In [238]:
features.shape

torch.Size([10011, 37845])

In [ ]:
# Takes too long !!
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=False)

In [240]:
spectral  = Spectral(30)

In [241]:
# Reduce feature space dimension
features = torch.Tensor(spectral.fit_transform(biadjacency))

In [ ]:
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=True)

In [243]:
eval_model(gnn, graph, features, labels, test)

0.6696606786427146

In [122]:
# Get random seeds
n_seeds = 1000
seeds = {i: labels.numpy()[i] for i in np.random.choice(range(len(labels)), n_seeds,  replace=False)}

dirichlet = DirichletClassifier()
labels_pred_diffusion = dirichlet.fit_transform(adjacency, seeds)
f1_score(np.array(labels[test]), np.array(labels_pred_diffusion[test]), average='micro')

0.6926147704590818

## To do

* What are the most similar articles to **Vincent van Gogh** in terms of cosine similarity in the embedding learned by the GNN?
* List the 20 closest articles to **Cat** and **Dog** in terms of cosine similarity in the embedding space (you may consider the mean vector of **Cat** and **Dog**).
* List the 20 closest articles to the category **Mathematics** in terms of cosine similarity in the embedding space.

In [244]:
def train_model(gnn, graph, features, labels, train, val, n_epochs=100, verbose=False):
    
    optimizer = torch.optim.Adam(gnn.parameters(), lr=0.01)
    
    gnn.train()
    scores = []
    
    for t in range(n_epochs):   
        
        # forward
        output = gnn(graph, features)
        logp = nn.functional.log_softmax(output, 1)
        loss = nn.functional.nll_loss(logp[train], labels[train])

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # evaluation
        score = eval_model(gnn, graph, features, labels, val)
        scores.append(score)
        
        if verbose and t % 5 == 0:
            print("Epoch {:02d} | Loss {:.3f} | Score {:.3f}".format(t, loss.item(), score))
            
    return output # Returns output

In [245]:
gnn = init_model(GNN, features, labels)
embedding = train_model(gnn, graph, features, labels, train, val, verbose=False)

In [248]:
embedding.shape

torch.Size([10011, 11])

In [249]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [246]:
emb = embedding.detach().numpy()

In [247]:
names = wikivitals.names

In [127]:
idx = np.where(names=='Vincent van Gogh')[0][0]
idx

9540

In [128]:
similarities = emb.dot(emb[idx])
print(names[np.argsort(-similarities)[:10]])

['Svetlana Alexievich' 'Snow Country' 'Kazuo Ishiguro' 'Naguib Mahfouz'
 'Saul Bellow' 'V. S. Naipaul' 'Duino Elegies' 'Ivan Bunin' 'T. S. Eliot'
 'Ismail Kadare']


In [129]:
idx = np.where(names=='Pablo Picasso')[0][0]
similarities = emb.dot(emb[idx])
print(names[np.argsort(-similarities)[:10]])

['Svetlana Alexievich' 'Snow Country' 'Kazuo Ishiguro' 'Naguib Mahfouz'
 'Saul Bellow' 'Duino Elegies' 'V. S. Naipaul' 'Leaves of Grass'
 'To the Lighthouse' 'Ismail Kadare']


In [130]:
{i:name for i, name in enumerate(names) if name == 'Cat' or name == 'Dog'}

{1497: 'Cat', 2468: 'Dog'}

In [131]:
target = [1497, 2468]

In [132]:
sim = emb.dot(np.mean(emb[target], axis=0))
print(names[np.argsort(-sim)[:20]])

['Puya (plant)' 'Solidago' 'Anthurium' 'Liliaceae' 'Bromeliaceae'
 'Iridaceae' 'Araceae' 'Apocynaceae' 'Gladiolus' 'Campanulaceae'
 'Hypericum' 'Primulaceae' 'Amanita phalloides' 'True frog' 'Hylidae'
 'Melastomataceae' 'Mushroom poisoning' 'Agaricus' 'Root canal'
 'Vascular cambium']


In [133]:
{i:name for i,name in enumerate(names) if name == 'Mathematics'}

{5735: 'Mathematics'}

In [134]:
categories = wikivitals.names_labels

In [135]:
categories[5]

'Mathematics'

In [136]:
label = 5

In [137]:
sim = emb.dot(np.mean(emb[labels == label], axis=0))
print(names[np.argsort(-sim)[:20]])

['Carl Gustav Jacob Jacobi' "Cauchy's integral formula"
 'Algebraic number field' 'Del' 'Implicit function'
 'Alexander Grothendieck' 'Charles Hermite' 'Niels Henrik Abel'
 'Partial derivative' 'Joseph Fourier' "Euler's formula"
 'Module (mathematics)' 'Commutative ring' 'Riemann hypothesis'
 'Multiple integral' 'Prime number theorem' "Laplace's equation"
 "Euler's identity" 'Homological algebra' 'Inverse function']
